In [1]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from anomaly_qwen2_5_vl import Anomaly_Qwen2_5_VLForConditionalGeneration, Qwen2_5_VLProcessor
from qwen_vl_utils import process_vision_info

best_ckpt_path = './ckpts/step004800.pth'
print(f"\n正在加载最佳权重: {best_ckpt_path}")

base = Anomaly_Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Geo/Anomaly_Qwen2.5-VL-7B-Instruct",
    torch_dtype="auto",
    device_map="auto",
)
processor = Qwen2_5_VLProcessor.from_pretrained("Geo/Anomaly_Qwen2.5-VL-7B-Instruct")
print(base.model.anomaly_maps_decoder)
print(base.model.prompt_learner)

state_dict = torch.load(best_ckpt_path, map_location="cpu")["state_dict"]
missing, unexpected = base.model.anomaly_maps_decoder.load_state_dict(state_dict, strict=False)
print("[Info] missing:", missing, '|', "[Info] unexpected:", unexpected)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Normal batch: {'input_ids': torch.Size([1820, 21]), 'attention_mask': torch.Size([1820, 21])} 
Abnormal batch: {'input_ids': torch.Size([1820, 21]), 'attention_mask': torch.Size([1820, 21])}

正在加载最佳权重: ./ckpts/step004800.pth


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of the model checkpoint at Geo/Anomaly_Qwen2.5-VL-7B-Instruct were not used when initializing Anomaly_Qwen2_5_VLForConditionalGeneration: ['model.language_model.anomaly_maps_decoder.fuse_weights', 'model.language_model.anomaly_maps_decoder.mlps.0.model.0.bias', 'model.language_model.anomaly_maps_decoder.mlps.0.model.0.weight', 'model.language_model.anomaly_maps_decoder.mlps.0.model.2.bias', 'model.language_model.anomaly_maps_decoder.mlps.0.model.2.weight', 'model.language_model.anomaly_maps_decoder.mlps.1.model.0.bias', 'model.language_model.anomaly_maps_decoder.mlps.1.model.0.weight', 'model.language_model.anomaly_maps_decoder.mlps.1.model.2.bias', 'model.language_model.anomaly_maps_decoder.mlps.1.model.2.weight', 'model.language_model.anomaly_maps_decoder.mlps.2.model.0.bias', 'model.language_model.anomaly_maps_decoder.mlps.2.model.0.weight', 'model.language_model.anomaly_maps_decoder.mlps.2.model.2.bias', 'model.language_model.anomaly_maps_decoder.mlps.2.model.2.weight'

AnomalyHeatmap(
  (mlps): ModuleList(
    (0-4): 5 x MLP(
      (model): Sequential(
        (0): Linear(in_features=1280, out_features=1024, bias=True)
        (1): ReLU(inplace=True)
        (2): Linear(in_features=1024, out_features=3584, bias=True)
      )
    )
  )
)
PromptLearner(
  (backbone): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): Conv2d(48, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
  )
  (region_attn_conv): Conv2d(96, 9, kernel_size=(1, 1), stride=(1, 1))
  (region_fc): Linear(in_features=96, out_features=3584, bias=True)
  (global_fc): Linear(in_features=96, out_features=3584, bias=True)
)
[Info] missing: [] | [Info] unexpected: []


In [2]:
# 只训练 prompt_learner
for p in base.parameters():
    p.requires_grad_(False)
for p in base.model.prompt_learner.parameters():
    p.requires_grad_(True)